In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #是否用gpu训练
BATCH_SIZE = 64 # 每批次处理的数据
EPOCHS = 10 #训练数据集的轮次
d2l.use_svg_display()

In [2]:
batch_size = 256
def get_dataloader_workers():
    """使用四个进程来读取数据"""
    return 4

In [3]:
"""加载数据并封装的集合"""
def load_data_fashion_mnist(batch_size, resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0,transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="../data", train=False, transform=trans, download=True)

    return (data.DataLoader(mnist_train,batch_size, shuffle=True,num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test,batch_size, shuffle=True,num_workers=get_dataloader_workers()))
 

In [4]:
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5) # 1灰度图片的通道 10输出通道 5：kernel卷积核
        self.conv2 = nn.Conv2d(10,20,3) #10:输入通道 20：输出通道 3：kernel
        #全连接层
        self.fc1 =nn.Linear(20*10*10,500) #20*10*10:输入通道，500：输出通道
        self.fc2 = nn.Linear(500,10)#500:输入通道，10：输出通道

    def forward(self,x):
        input_size = x.size(0) #batch_size x 1(灰度） x 28 x 28（像素）
        x = self.conv1(x) #输入：batch*1*28*28，输出：batch*10*24*24（28-5+1=24）
        x = F.relu(x)#激活函数 保持形状不变 输出：batch*10*24*24
        x = F.max_pool2d(x,2,2)#池化层 ,把x变成分块矩阵，找其中的最大值（提取图片最明显的特征，忽略小细节）
                                #输入batch*10*24*24 输出：batch*10*12*12 （步长为2，减半）
        x = self.conv2(x) #输入batch*10*12*12 输出：batch*20*10*10（12-3+1=10）
        x = F.relu(x)
        
        x = x.view(input_size,-1)# 拉平，-1自动计算维度，20*10*10=2000 
        x = self.fc1(x) #输入：batch*2000 输出：batch*500（全连接层定义500）
        x = F.relu(x) # 保持shape不变

        x = self.fc2(x) #输入：Batch*500 输出：batch*10

        output = F.log_softmax(x,dim=1) #计算分类后，每个数字的概率值
        return output
#定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())#优化器，用adam每个训练参数的学习率可以自适应变化


In [5]:
#定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    #模型训练
    model.train()
    for batch_index,(data,target) in enumerate(train_loader):
        #部署到device上
        data, target = data.to(device), target.to(device)
        #梯度初始化
        optimizer.zero_grad()
        #预测 训练后的结果
        output = model(data)
        #计算损失
        loss = F.cross_entropy(output,target)#交叉熵损失，针对多分类任务
        #反向传播 更新权重，神经元参数
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 ==0:
            print("Train Epoch:{}\t Loss:{:.6f}\n".format(epoch,loss.item()))
        

In [6]:
#定义测试方法
def test_model(model, device, test_loader):
    #模型验证
    model.eval()
    #正确率
    correct = 0.0
    #测试损失
    test_loss = 0.0
    with torch.no_grad(): #不会计算梯度，也不进行反向传播
        for data, target in test_loader:
            #部署
            data,target = data.to(device), target.to(device)
            #测试数据
            output = model(data)
            #计算损失
            test_loss += F.cross_entropy(output,target).item()
            #找出最大值下标
            pred = output.argmax(dim=1)
            #累计正确率
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        accuracy = 100.0*correct / len(test_loader.dataset)
        print("Test_Average Loss:{:.4f}\t Accuracy:{:.3f}%\n".format(test_loss,accuracy))

In [7]:
#调用方法，输出结果
train_loader,test_loader=load_data_fashion_mnist(BATCH_SIZE)
for epoch in range(1,EPOCHS+1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_result = test_model(model,DEVICE,test_loader)

Train Epoch:1	 Loss:2.294102

Test_Average Loss:0.0059	 Accuracy:86.630%

Train Epoch:2	 Loss:0.351351

Test_Average Loss:0.0051	 Accuracy:88.290%

Train Epoch:3	 Loss:0.291299

Test_Average Loss:0.0044	 Accuracy:89.850%

Train Epoch:4	 Loss:0.292003

Test_Average Loss:0.0044	 Accuracy:89.840%

Train Epoch:5	 Loss:0.297476

Test_Average Loss:0.0042	 Accuracy:90.290%

Train Epoch:6	 Loss:0.151213

Test_Average Loss:0.0044	 Accuracy:90.040%

Train Epoch:7	 Loss:0.168174

Test_Average Loss:0.0041	 Accuracy:91.490%

Train Epoch:8	 Loss:0.310686

Test_Average Loss:0.0044	 Accuracy:90.370%

Train Epoch:9	 Loss:0.132526

Test_Average Loss:0.0043	 Accuracy:91.120%

Train Epoch:10	 Loss:0.053007

Test_Average Loss:0.0049	 Accuracy:91.020%

